In [0]:
#Team 2 Notebook First Commit #
#Commit 2
aqi_county_df_2020 = spark.read.csv("/Volumes/workspace/2235_team2/aqi/annual_aqi_by_county_2020.csv", header=True, inferSchema=True)
aqi_county_df_2020.show(5)

In [0]:
aqi_county_df_2021 = spark.read.csv("/Volumes/workspace/2235_team2/aqi/annual_aqi_by_county_2021.csv", header=True, inferSchema=True)
aqi_county_df_2021.show(5)

In [0]:
aqi_county_df_2020.printSchema()

In [0]:
aqi_county_df_2021.printSchema()

In [0]:
aqi_county_df_2022 = spark.read.csv("/Volumes/workspace/2235_team2/aqi/annual_aqi_by_county_2022.csv", header=True, inferSchema=True)
aqi_county_df_2023 = spark.read.csv("/Volumes/workspace/2235_team2/aqi/annual_aqi_by_county_2023.csv", header=True, inferSchema=True)
aqi_county_df_2024 = spark.read.csv("/Volumes/workspace/2235_team2/aqi/annual_aqi_by_county_2024.csv", header=True, inferSchema=True)
aqi_county_df_2022.show(5)

In [0]:
aqi_county_df_2023.show(5)

In [0]:
aqi_county_df_2024.count()

In [0]:
aqi_county_df = aqi_county_df_2020.union(aqi_county_df_2021).union(aqi_county_df_2022).union(aqi_county_df_2023).union(aqi_county_df_2024)
aqi_county_df.show(5)
aqi_county_df.count()

In [0]:
aqi_county_df.show(5)

In [0]:
aqi_county_df.printSchema()

In [0]:
aqi_cbsa_df_2020 = spark.read.csv("/Volumes/workspace/2235_team2/aqi/annual_aqi_by_cbsa_2020.csv", header=True, inferSchema=True)
aqi_cbsa_df_2021 = spark.read.csv("/Volumes/workspace/2235_team2/aqi/annual_aqi_by_cbsa_2021.csv", header=True, inferSchema=True)
aqi_cbsa_df_2022 = spark.read.csv("/Volumes/workspace/2235_team2/aqi/annual_aqi_by_cbsa_2022.csv", header=True, inferSchema=True)
aqi_cbsa_df_2023 = spark.read.csv("/Volumes/workspace/2235_team2/aqi/annual_aqi_by_cbsa_2023.csv", header=True, inferSchema=True)
aqi_cbsa_df_2024 = spark.read.csv("/Volumes/workspace/2235_team2/aqi/annual_aqi_by_cbsa_2024.csv", header=True, inferSchema=True)


In [0]:
aqi_cbsa_df_2020.printSchema()

In [0]:
aqi_cbsa_df_2021.printSchema()

In [0]:
aqi_cbsa_df_2020.show(5)
aqi_cbsa_df_2021.show(5)
aqi_cbsa_df_2020.count()
aqi_cbsa_df_2021.count()

In [0]:
aqi_cbsa_df = aqi_cbsa_df_2020.union(aqi_cbsa_df_2021).union(aqi_cbsa_df_2022).union(aqi_cbsa_df_2023).union(aqi_cbsa_df_2024)
aqi_cbsa_df.show(5)
aqi_cbsa_df.count()

In [0]:
aqi_cbsa_df.count()

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
#importing cbsa & county ref file
cbsa_county_ref = spark.read.csv("/Volumes/workspace/2235_team2/aqi/cbsa_county.csv", header=True, inferSchema=True)
cbsa_county_ref.show(5)

In [0]:
cbsa_county_ref.printSchema()

In [0]:
#Joining with CBSA & County Ref
#aqi_cbsa_joined_df = aqi_cbsa_df.join(cbsa_county_ref,aqi_cbsa_df.CBSA == cbsa_county_ref['CBSA Title'] , how='inner')
#aqi_cbsa_joined_df.show(2)

In [0]:
from pyspark.sql.functions import split, col

# Assuming your DataFrame is named df
df_split = aqi_cbsa_df.withColumn("County", split(col("CBSA"), ", ")[0]) \
             .withColumn("State", split(col("CBSA"), ", ")[1])

# Select the new columns to display the result
df_split.show(2)

In [0]:
aqi_cbsa_df = df_split
aqi_cbsa_df.show(2)

In [0]:
aqi_cbsa_df.printSchema()

In [0]:
from pyspark.sql.functions import trim

# Assume df is your Spark DataFrame and 'column_name' is the column you want to trim
df_trimmed = aqi_cbsa_df.withColumn('County', trim(aqi_cbsa_df['County']))
df_trimmed = aqi_cbsa_df.withColumn('State', trim(aqi_cbsa_df['State']))
df_trimmed.show(5)

In [0]:
#Importing FIPS Code for County and State
fips_code_county = spark.read.csv("/Volumes/workspace/2235_team2/aqi/FIPS_Codes_County.csv", header=True, inferSchema=True)
fips_code_state = spark.read.csv("/Volumes/workspace/2235_team2/aqi/FIPS_Codes_State.csv", header=True, inferSchema=True)

In [0]:
fips_code_county = fips_code_county.withColumn('County', trim(fips_code_county['County']))

In [0]:
fips_code_county.show(5)

In [0]:
fips_code_county.printSchema()

In [0]:
fips_code_state.show(5)

In [0]:
fips_code_state.printSchema()

In [0]:
fips_code_state = fips_code_state.withColumn('State', lower(trim(fips_code_state['State'])))


In [0]:
fips_code_state = fips_code_state.withColumn('State Code', trim(fips_code_state['State Code']))

In [0]:
fips_code_state.show(5)

In [0]:
aqi_cbsa_df = df_trimmed
aqi_cbsa_df.show(5)

In [0]:
aqi_cbsa_merged_df = aqi_cbsa_df.join(fips_code_county,aqi_cbsa_df.County == fips_code_county.County, how='inner')
aqi_cbsa_merged_df.show(5)

In [0]:
# Assume df is your Spark DataFrame and 'column_name' is the column to check for nulls
from pyspark.sql.functions import col
#aqi_cbsa_merged_nulls = aqi_cbsa_merged_df.filter(aqi_cbsa_merged_df['FIPS County Code'].isNull())
null_count = aqi_cbsa_merged_df.filter(col('FIPS County Code').isNull()).count()
print(f"Number of nulls: {null_count}")

In [0]:
aqi_cbsa_merged_df.count()

In [0]:
aqi_cbsa_merged_final_df = aqi_cbsa_merged_df.join(fips_code_state,aqi_cbsa_merged_df.State ==fips_code_state['State Code'] , how='inner')
aqi_cbsa_merged_final_df.show(5)

In [0]:
null_count = aqi_cbsa_merged_final_df.filter(col('FIPS State Code').isNull()).count()
print(f"Number of nulls: {null_count}")

In [0]:
aqi_county_df.printSchema()

In [0]:

# Assume df is your Spark DataFrame and 'column_name' is the column you want to trim
aqi_county_df_trimmed = aqi_county_df.withColumn('County', trim(aqi_county_df['County']))
aqi_county_df_trimmed = aqi_county_df.withColumn('State', lower(trim(aqi_county_df['State'])))
aqi_county_df_trimmed.show(5)

In [0]:
aqi_county_df = aqi_county_df_trimmed
aqi_county_df.show(2)

In [0]:
aqi_county_merged_df = aqi_county_df.join(fips_code_county,aqi_county_df.County == fips_code_county.County, how='inner')
aqi_county_merged_df.show(2)

In [0]:
aqi_county_merged_df.count()

In [0]:
null_count = aqi_county_merged_df.filter(col('FIPS County Code').isNull()).count()
print(f"Number of nulls: {null_count}")

In [0]:
aqi_county_merged_final_df = aqi_county_merged_df.join(fips_code_state,aqi_county_merged_df.State ==fips_code_state['State'] , how='inner')
aqi_county_merged_final_df.show(4)

In [0]:
aqi_county_merged_final_df.count()

In [0]:
null_count = aqi_county_merged_final_df.filter(col('FIPS State Code').isNull()).count()
print(f"Number of nulls: {null_count}")

In [0]:
aqi_cbsa_df = aqi_cbsa_merged_final_df
aqi_county_df = aqi_county_merged_final_df

In [0]:
aqi_cbsa_df.show(5)

In [0]:
aqi_county_df.show(5)

In [0]:
aqi_cbsa_df.count()

In [0]:
aqi_county_df.count()